In [1]:
import sys
import os

sys.path.append(os.path.abspath('../package'))
%load_ext autoreload
%autoreload 2

from functions.data_loader import load_excel_data
file = "../../data/BASE DE DATOS PCC_cleaned.xlsx"
df_char = load_excel_data(file)

Archivo cargado correctamente desde ../../data/BASE DE DATOS PCC_cleaned.xlsx


In [15]:

from functions.data_preparation import prepare_regression_data

features = {
    'pedido': 'numeric',
    'op': 'numeric',
    'referencia': 'categorical',
    'seccion': 'categorical',
    'maquina': 'categorical',
    'proceso': 'categorical',
    'usuario': 'categorical',
    'fabricadas': 'numeric',
    'hora_inicio': 'numeric',
    'dia_semana_inicio': 'categorical',
    'estado': 'categorical'
}

X_train, X_test, y_train, y_test, encoder, scaler = prepare_regression_data(df_char,features,"duracion_min")

Data prepared: 14608 training samples, 3653 test samples.
Categorical features encoded: ['referencia', 'seccion', 'maquina', 'proceso', 'usuario', 'dia_semana_inicio', 'estado']


In [18]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train)

#Get the explained variance ratio for each component
var_exp_acum = np.cumsum(pca.explained_variance_ratio_)
#Print the explained variance ratio
print("Varianza explicada acumulada por cada componente:")
for i, var in enumerate(var_exp_acum):
    print(f"Componente {i+1}: {var:.4f}")
threshold = 0.999
#Select the number of components that explain at least 95% of the variance
n_components = np.argmax(var_exp_acum >=threshold) + 1
print(f"Se eligen {n_components} componentes para explicar el {round(threshold*100,2)}% de la varianza")

Varianza explicada acumulada por cada componente:
Componente 1: 0.9978
Componente 2: 0.9994
Componente 3: 0.9997
Componente 4: 0.9998
Componente 5: 0.9999
Componente 6: 1.0000
Componente 7: 1.0000
Componente 8: 1.0000
Componente 9: 1.0000
Componente 10: 1.0000
Componente 11: 1.0000
Se eligen 2 componentes para explicar el 99.9% de la varianza


In [19]:
#Adjust PCA with the selected number of components
pca_final = PCA(n_components=n_components)
X_train_pca = pca_final.fit_transform(X_train)
X_test_pca = pca_final.transform(X_test)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Lasso': {
        'model': Lasso(),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]
        }
    },
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
        }
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5]
        }
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {
            'n_neighbors': [1,3, 5, 7, 9, 11,15,20,25,30,35,40],
            'weights': ['uniform', 'distance'],
            'p': [1, 2]  # 1 = Manhattan, 2 = Euclidean
        }
    }
}

In [21]:
from functions.model_evaluation import evaluate_regression_models
# Filter X_train and X_test to keep only features selected by OLS

tuned_results, summary_metrics, all_grid,best_models = evaluate_regression_models(
    models,
    X_train_pca, y_train,
    X_test=X_test_pca,
    y_test=y_test,
    scoring="neg_root_mean_squared_error",
    cv=5
)

Evaluating: LinearRegression
Best Params                 {}
RMSE CV              53.544747
MAE CV                39.36807
R2 CV                 0.005342
RMSE Test            54.554741
MAE Test             40.271309
R2 Test               0.008072
Training Time (s)     3.112656
dtype: object


Evaluating: Lasso
Best Params          {'alpha': 10.0}
RMSE CV                    53.535504
MAE CV                      39.36289
R2 CV                       0.005686
RMSE Test                  54.547829
MAE Test                   40.265374
R2 Test                     0.008323
Training Time (s)           2.247326
dtype: object


Evaluating: Ridge
Best Params          {'alpha': 100.0}
RMSE CV                     53.544746
MAE CV                      39.368069
R2 CV                        0.005342
RMSE Test                    54.55474
MAE Test                    40.271308
R2 Test                      0.008072
Training Time (s)            0.075742
dtype: object


Evaluating: RandomForest
Best Params   